In [262]:
import pandas as pd
import numpy as np
import random
import os
from glob import glob
import matplotlib.pyplot as plt
import cv2
import keras
import matplotlib.pylab as plt
import plotly.graph_objects as go
import pandas as pd
import keras
from keras.callbacks import ModelCheckpoint, ReduceLROnPlateau, TerminateOnNaN, CSVLogger
from tensorflow.keras.metrics import AUC
from glob import glob
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.layers import (Conv2D, GlobalMaxPool2D, GlobalAveragePooling2D,
                                     Dropout, Flatten, Dense, Activation, Dropout,
                                     BatchNormalization, MaxPooling2D)
from tensorflow.keras.applications.mobilenet import MobileNet
from keras.applications.densenet import DenseNet121, preprocess_input
from tensorflow.keras import regularizers, optimizers
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.callbacks import (ModelCheckpoint, LearningRateScheduler, 
                                        EarlyStopping, CSVLogger, TerminateOnNaN,
                                       ReduceLROnPlateau)

pd.set_option('display.width', 500)
pd.set_option('display.max_columns', 100)
pd.set_option('display.max_rows', 100)
pd.set_option('display.notebook_repr_html', True)

In [269]:
path_to_frame = 'K:\DiplomZalevskiy\Data_Entry_2017_v2020.csv'
# images = glob(os.path.join(path_images,'*.png'))
len([file for file in os.listdir("K:\DiplomZalevskiy\data\images") if file.endswith('.png')])


112120

In [270]:
origin_df = pd.read_csv(path_to_frame)

In [271]:
def rewrite_df(path_to_frame):
    result_df = pd.read_csv(path_to_frame)
    result_df = result_df.drop(
        ['Patient Age','Patient Gender','View Position',
         'OriginalImage[Width','Height]','OriginalImagePixelSpacing[x','y]'],axis=1)
    result_df.columns = ['filename','disease','followUP','patientID']
    result_df['path'] = 'K:/DiplomZalevskiy/data/images/' + result_df['filename'].replace(' ','')
    return result_df

def categorical_df(dataframe):
    disease_labels = ['Atelectasis','Cardiomegaly','Consolidation','Edema','Effusion','Emphysema',
                      'Fibrosis','Hernia','Infiltration','Mass','No Finding','Nodule','Pleural_Thickening',
                      'Pneumonia','Pneumothorax']
    for i in disease_labels:
        dataframe[i] = dataframe.disease.map(lambda result: 1 if i in result else 0)
    dataframe['disease_vector'] = dataframe[disease_labels].apply(lambda x: x.tolist(),axis=1)
    return dataframe

In [272]:
test_df = rewrite_df(path_to_frame)
#test_df[0:15]
disease_labels = ['Atelectasis','Cardiomegaly','Consolidation','Edema','Effusion','Emphysema',
                      'Fibrosis','Hernia','Infiltration','Mass','No Finding','Nodule','Pleural_Thickening',
                      'Pneumonia','Pneumothorax']
final_df = categorical_df(test_df)
info_origin_dict = final_df[disease_labels].sum().to_dict()
#info_origin_dict
final_df

,filename,disease,followUP,patientID,path,Atelectasis,Cardiomegaly,Consolidation,Edema,Effusion,Emphysema,Fibrosis,Hernia,Infiltration,Mass,No Finding,Nodule,Pleural_Thickening,Pneumonia,Pneumothorax,disease_vector
0,00000001_000.png,Cardiomegaly,0,1,K:/DiplomZalevskiy/data/images/00000001_000.png,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,"[0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]"
1,00000001_001.png,Cardiomegaly|Emphysema,1,1,K:/DiplomZalevskiy/data/images/00000001_001.png,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,"[0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0]"
2,00000001_002.png,Cardiomegaly|Effusion,2,1,K:/DiplomZalevskiy/data/images/00000001_002.png,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,"[0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]"
3,00000002_000.png,No Finding,0,2,K:/DiplomZalevskiy/data/images/00000002_000.png,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0]"
4,00000003_001.png,Hernia,0,3,K:/DiplomZalevskiy/data/images/00000003_001.png,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,"[0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0]"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
112115,00030801_001.png,Mass|Pneumonia,1,30801,K:/DiplomZalevskiy/data/images/00030801_001.png,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0]"
112116,00030802_000.png,No Finding,0,30802,K:/DiplomZalevskiy/data/images/00030802_000.png,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0]"
112117,00030803_000.png,No Finding,0,30803,K:/DiplomZalevskiy/data/images/00030803_000.png,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0]"
112118,00030804_000.png,No Finding,0,30804,K:/DiplomZalevskiy/data/images/00030804_000.png,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0]"


In [273]:
final_df[final_df['Pneumonia']!=1][0:15]

,filename,disease,followUP,patientID,path,Atelectasis,Cardiomegaly,Consolidation,Edema,Effusion,Emphysema,Fibrosis,Hernia,Infiltration,Mass,No Finding,Nodule,Pleural_Thickening,Pneumonia,Pneumothorax,disease_vector
0,00000001_000.png,Cardiomegaly,0,1,K:/DiplomZalevskiy/data/images/00000001_000.png,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,"[0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]"
1,00000001_001.png,Cardiomegaly|Emphysema,1,1,K:/DiplomZalevskiy/data/images/00000001_001.png,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,"[0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0]"
2,00000001_002.png,Cardiomegaly|Effusion,2,1,K:/DiplomZalevskiy/data/images/00000001_002.png,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,"[0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]"
3,00000002_000.png,No Finding,0,2,K:/DiplomZalevskiy/data/images/00000002_000.png,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0]"
4,00000003_001.png,Hernia,0,3,K:/DiplomZalevskiy/data/images/00000003_001.png,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,"[0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0]"
5,00000003_002.png,Hernia,1,3,K:/DiplomZalevskiy/data/images/00000003_002.png,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,"[0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0]"
6,00000003_003.png,Hernia|Infiltration,2,3,K:/DiplomZalevskiy/data/images/00000003_003.png,0,0,0,0,0,0,0,1,1,0,0,0,0,0,0,"[0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0]"
7,00000003_004.png,Hernia,3,3,K:/DiplomZalevskiy/data/images/00000003_004.png,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,"[0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0]"
8,00000003_005.png,Hernia,4,3,K:/DiplomZalevskiy/data/images/00000003_005.png,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,"[0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0]"
9,00000003_006.png,Hernia,5,3,K:/DiplomZalevskiy/data/images/00000003_006.png,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,"[0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0]"


In [257]:
illness_df = final_df[final_df['No Finding'] != 1] 
illness_df['Normal']=illness_df['No Finding']
illness_df

C:\Users\Yaroslavel\AppData\Local\Temp\ipykernel_22224\1487900478.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  illness_df['Normal']=illness_df['No Finding']


,filename,disease,followUP,patientID,path,Atelectasis,Cardiomegaly,Consolidation,Edema,Effusion,Emphysema,Fibrosis,Hernia,Infiltration,Mass,No Finding,Nodule,Pleural_Thickening,Pneumonia,Pneumothorax,disease_vector,Normal
0,00000001_000.png,Cardiomegaly,0,1,K:/DiplomZalevskiy/data/images/00000001_000.png,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,"[0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]",0
1,00000001_001.png,Cardiomegaly|Emphysema,1,1,K:/DiplomZalevskiy/data/images/00000001_001.png,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,"[0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0]",0
2,00000001_002.png,Cardiomegaly|Effusion,2,1,K:/DiplomZalevskiy/data/images/00000001_002.png,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,"[0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]",0
4,00000003_001.png,Hernia,0,3,K:/DiplomZalevskiy/data/images/00000003_001.png,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,"[0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0]",0
5,00000003_002.png,Hernia,1,3,K:/DiplomZalevskiy/data/images/00000003_002.png,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,"[0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0]",0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
112097,00030786_006.png,Consolidation,7,30786,K:/DiplomZalevskiy/data/images/00030786_006.png,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,"[0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]",0
112100,00030789_000.png,Infiltration,0,30789,K:/DiplomZalevskiy/data/images/00030789_000.png,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,"[0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0]",0
112106,00030793_000.png,Mass|Nodule,0,30793,K:/DiplomZalevskiy/data/images/00030793_000.png,0,0,0,0,0,0,0,0,0,1,0,1,0,0,0,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0]",0
112108,00030795_000.png,Pleural_Thickening,0,30795,K:/DiplomZalevskiy/data/images/00030795_000.png,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0]",0


In [258]:
# выбросили те заболевания где нет чистой hernia и пневмонии
illness_df= illness_df[ illness_df['Hernia'] !=1]
illness_df= illness_df[ illness_df['Pneumonia'] !=1]

In [ ]:
illness_df.sum()

In [ ]:
illness_df = illness_df.drop(['No Finding','Hernia','Pneumonia'], axis=1)
final_df = final_df.drop(['Hernia','Pneumonia'], axis=1)

In [ ]:
illness_df

In [ ]:
final_df['Normal'] = final_df['No Finding']
final_df = final_df.drop(['No Finding'],axis=1)


In [ ]:
# здоровые пациенты, берем 4000 записей
normal_df = final_df[final_df['Normal'] == 1].loc[0:7100,:] 

#пациенты с эффузией
effusion_df = final_df[final_df['Effusion'] == 1].loc[0:40000,:]
effusion_df

#инфильрат
infiltration_df = final_df[final_df['Infiltration'] == 1].loc[0:22000,:]

#Ателекстаз
atelectasis_df = final_df[final_df['Atelectasis'] == 1].loc[0:33000,:]
atelectasis_df

In [ ]:
illness_df.disease.value_counts()[0:15]

In [ ]:
# удаляем из illnes
illness_df = illness_df.drop(index=illness_df[illness_df['Effusion']==1].index,axis=0)
illness_df = illness_df.drop(index=illness_df[illness_df['Infiltration']==1].index,axis=0)
illness_df = illness_df.drop(index=illness_df[illness_df['Atelectasis']==1].index,axis=0)

In [ ]:
illness_df.sum()

In [ ]:
illness_df

In [159]:
info_origin_dict

{'Atelectasis': 11559,
 'Cardiomegaly': 2776,
 'Consolidation': 4667,
 'Edema': 2303,
 'Effusion': 13317,
 'Emphysema': 2516,
 'Fibrosis': 1686,
 'Hernia': 227,
 'Infiltration': 19894,
 'Mass': 5782,
 'No Finding': 60361,
 'Nodule': 6331,
 'Pleural_Thickening': 3385,
 'Pneumonia': 1431,
 'Pneumothorax': 5302}

In [140]:
balanced_df = illness_df.append([normal_df,atelectasis_df,infiltration_df,effusion_df])
path = balanced_df['path']
balanced_df.sum()

C:\Users\Yaroslavel\AppData\Local\Temp\ipykernel_22224\814520242.py:1: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  balanced_df = illness_df.append([normal_df,atelectasis_df,infiltration_df,effusion_df])


filename              00000001_000.png00000001_001.png00000004_000.p...
disease               CardiomegalyCardiomegaly|EmphysemaMass|NoduleC...
followUP                                                         298827
patientID                                                     270953747
path                  K:/DiplomZalevskiy/data/images/00000001_000.pn...
Atelectasis                                                        4616
Cardiomegaly                                                       1854
Consolidation                                                      2741
Edema                                                              1157
Effusion                                                           5328
Emphysema                                                          1804
Fibrosis                                                           1278
Infiltration                                                       5152
Mass                                                            

In [141]:
#30057 rows × 18 columns
balanced_df

,filename,disease,followUP,patientID,path,Atelectasis,Cardiomegaly,Consolidation,Edema,Effusion,Emphysema,Fibrosis,Infiltration,Mass,Nodule,Pleural_Thickening,Pneumothorax,Normal
0,00000001_000.png,Cardiomegaly,0,1,K:/DiplomZalevskiy/data/images/00000001_000.png,0,1,0,0,0,0,0,0,0,0,0,0,0
1,00000001_001.png,Cardiomegaly|Emphysema,1,1,K:/DiplomZalevskiy/data/images/00000001_001.png,0,1,0,0,0,1,0,0,0,0,0,0,0
12,00000004_000.png,Mass|Nodule,0,4,K:/DiplomZalevskiy/data/images/00000004_000.png,0,0,0,0,0,0,0,0,1,1,0,0,0
23,00000008_000.png,Cardiomegaly,0,8,K:/DiplomZalevskiy/data/images/00000008_000.png,0,1,0,0,0,0,0,0,0,0,0,0,0
25,00000008_002.png,Nodule,2,8,K:/DiplomZalevskiy/data/images/00000008_002.png,0,0,0,0,0,0,0,0,0,1,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
39959,00010443_001.png,Effusion,1,10443,K:/DiplomZalevskiy/data/images/00010443_001.png,0,0,0,0,1,0,0,0,0,0,0,0,0
39979,00010447_009.png,Consolidation|Effusion,9,10447,K:/DiplomZalevskiy/data/images/00010447_009.png,0,0,1,0,1,0,0,0,0,0,0,0,0
39994,00010447_024.png,Effusion|Infiltration,24,10447,K:/DiplomZalevskiy/data/images/00010447_024.png,0,0,0,0,1,0,0,1,0,0,0,0,0
39996,00010447_026.png,Effusion,26,10447,K:/DiplomZalevskiy/data/images/00010447_026.png,0,0,0,0,1,0,0,0,0,0,0,0,0


In [245]:
#final_balanced_df =balanced_df.drop_duplicates('path') 
final_balanced_df

,filename,disease,followUP,patientID,path,Atelectasis,Cardiomegaly,Consolidation,Edema,Effusion,Emphysema,Fibrosis,Infiltration,Mass,Nodule,Pleural_Thickening,Pneumothorax,Normal
0,00000001_000.png,Cardiomegaly,0,1,K:/DiplomZalevskiy/data/images/00000001_000.png,0,1,0,0,0,0,0,0,0,0,0,0,0
1,00000001_001.png,Cardiomegaly|Emphysema,1,1,K:/DiplomZalevskiy/data/images/00000001_001.png,0,1,0,0,0,1,0,0,0,0,0,0,0
12,00000004_000.png,Mass|Nodule,0,4,K:/DiplomZalevskiy/data/images/00000004_000.png,0,0,0,0,0,0,0,0,1,1,0,0,0
23,00000008_000.png,Cardiomegaly,0,8,K:/DiplomZalevskiy/data/images/00000008_000.png,0,1,0,0,0,0,0,0,0,0,0,0,0
25,00000008_002.png,Nodule,2,8,K:/DiplomZalevskiy/data/images/00000008_002.png,0,0,0,0,0,0,0,0,0,1,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
39959,00010443_001.png,Effusion,1,10443,K:/DiplomZalevskiy/data/images/00010443_001.png,0,0,0,0,1,0,0,0,0,0,0,0,0
39979,00010447_009.png,Consolidation|Effusion,9,10447,K:/DiplomZalevskiy/data/images/00010447_009.png,0,0,1,0,1,0,0,0,0,0,0,0,0
39994,00010447_024.png,Effusion|Infiltration,24,10447,K:/DiplomZalevskiy/data/images/00010447_024.png,0,0,0,0,1,0,0,1,0,0,0,0,0
39996,00010447_026.png,Effusion,26,10447,K:/DiplomZalevskiy/data/images/00010447_026.png,0,0,0,0,1,0,0,0,0,0,0,0,0


In [250]:
disease_labels= ['Cardiomegaly','Emphysema','Effusion','Infiltration',
      'Mass','Nodule','Atelectasis','Pneumothorax',
      'Pleural_Thickening','Fibrosis','Edema','Consolidation']
disease_labels = sorted(disease_labels)
disease_labels.append('Normal')
disease_labels
final_balanced_df

,filename,disease,followUP,patientID,path,Atelectasis,Cardiomegaly,Consolidation,Edema,Effusion,Emphysema,Fibrosis,Infiltration,Mass,Nodule,Pleural_Thickening,Pneumothorax,Normal
0,00000001_000.png,Cardiomegaly,0,1,K:/DiplomZalevskiy/data/images/00000001_000.png,0,1,0,0,0,0,0,0,0,0,0,0,0
1,00000001_001.png,Cardiomegaly|Emphysema,1,1,K:/DiplomZalevskiy/data/images/00000001_001.png,0,1,0,0,0,1,0,0,0,0,0,0,0
12,00000004_000.png,Mass|Nodule,0,4,K:/DiplomZalevskiy/data/images/00000004_000.png,0,0,0,0,0,0,0,0,1,1,0,0,0
23,00000008_000.png,Cardiomegaly,0,8,K:/DiplomZalevskiy/data/images/00000008_000.png,0,1,0,0,0,0,0,0,0,0,0,0,0
25,00000008_002.png,Nodule,2,8,K:/DiplomZalevskiy/data/images/00000008_002.png,0,0,0,0,0,0,0,0,0,1,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
39959,00010443_001.png,Effusion,1,10443,K:/DiplomZalevskiy/data/images/00010443_001.png,0,0,0,0,1,0,0,0,0,0,0,0,0
39979,00010447_009.png,Consolidation|Effusion,9,10447,K:/DiplomZalevskiy/data/images/00010447_009.png,0,0,1,0,1,0,0,0,0,0,0,0,0
39994,00010447_024.png,Effusion|Infiltration,24,10447,K:/DiplomZalevskiy/data/images/00010447_024.png,0,0,0,0,1,0,0,1,0,0,0,0,0
39996,00010447_026.png,Effusion,26,10447,K:/DiplomZalevskiy/data/images/00010447_026.png,0,0,0,0,1,0,0,0,0,0,0,0,0


In [153]:
dirr = 'BalancedDF'
if not os.path.exists(dirr):
    os.makedirs(dirr)
final_balanced_df.to_pickle('BalancedDF/balanced_df.pkl')

In [ ]:
from sklearn.model_selection import train_test_split

# СОЗДАНИЕ ВЫБОРОК : ОБУЧЕНИЯ, ВАЛИДАЦИЯ, ТЕСТИРУЮЩАЯ

train_set, valid_set = train_test_split(final_balanced_df, 
                                        train_size=0.9, 
                                        random_state=42, 
                                        stratify=final_balanced_df.disease)
                                        #shuffle=True



# train_set, test_set = train_test_split(train_set, 
#                                        test_size=0.20, 
#                                        random_state=42, 
#                                        stratify=train_set.disease)

print('Разбиение данных:')
print('Обучающая выборка: ', train_set.shape[0])
print('Валидационная выборка: ', valid_set.shape[0])
print('Тестирующая выборка: ', test_set.shape[0])
print('Общее число записей:', final_balanced_df.shape[0])

In [1]:
import numpy as np
import cv2
def sharpering(img):
    kernel = np.array([[0, -1, 0], [-1, 5, -1], [0, -1, 0]])
    image_sharp = cv2.filter2D(src=img, ddepth=-1, kernel=kernel)
    return image_sharp

In [222]:
image_generator = ImageDataGenerator(rescale=1./255.,
                         samplewise_center=True,
                         samplewise_std_normalization=True,
                         horizontal_flip=False,
                         vertical_flip=False,
                         height_shift_range=0.1,
                         width_shift_range=0.1,
                         rotation_range=5,
                         shear_range=0.1,
                         fill_mode='reflect',
                         zoom_range=0.15,
                         preprocessing_function = sharpering)

train_generator = image_generator.flow_from_dataframe(
    dataframe=train_set,
    directory=None,
    x_col="path",
    y_col=disease_labels,
    target_size=(224,224),
    seed=42,
    class_mode='raw',
    batch_size=32)

valid_generator = image_generator.flow_from_dataframe(
    dataframe=valid_set,
    directory=None,
    x_col="path",
    y_col=disease_labels,
    target_size=(224,224),
    class_mode='raw',
    batch_size=128)

Found 20436 validated image filenames.
Found 2839 validated image filenames.


In [192]:
x_test,y_test = next(train_generator)

In [234]:
index_per_disease = {i:c for i,c in enumerate(disease_labels)}
len(index_per_disease)

13

In [224]:
# Коллбеки
model_checkpoint = ModelCheckpoint(
    filepath = 'mobilenet_cnn_{epoch:02d}_{val_loss:.2f}.hdf5',
    verbose = 1,
    monitor = 'val_loss',
    save_best_only= True)

reduce_lron = ReduceLROnPlateau(monitor='val_loss', 
                                factor=0.1, 
                                patience=3,
                                verbose=1, mode="min", 
                                min_lr=1e-5)

auc = AUC()
#model_checkpoint = ModelCheckpoint(..) # Описан ранее
#reduce_lron = ReduceLROnPlateau(...) # Описан ранее
ter_by_nan = TerminateOnNaN() 
logger = CSVLogger('mobienet_best_logger.cvs')
callbacks_lst = [reduce_lron,ter_by_nan,logger]

In [230]:
x_test.shape[1:]

(224, 224, 3)

In [241]:
# build model
from keras.layers import Input
input_img = Input(x_test.shape[1:])
base_model = DenseNet121(
                    input_shape = (224,224) + (3,),
                    include_top=False,
                    weights=None,
                    pooling='avg')

x = base_model.output
predictions = Dense(len(index_per_disease),activation='sigmoid',name='predictions')(x)
model = Model(base_model.input ,predictions)
#output = GlobalAveragePooling2D()(base_model.output)

# prediction = Dense(len(index_per_disease),activation='sigmoid',name="predictions")(output)

# myModel = Model(inputs=input_img,outputs = prediction)

# myModel.summary()

model.compile(optimizer='adam', loss = 'binary_crossentropy',
                      metrics=['categorical_accuracy', auc])
    
model.summary()

Model: "model_12"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_42 (InputLayer)          [(None, 224, 224, 3  0           []                               
                                )]                                                                
                                                                                                  
 zero_padding2d_54 (ZeroPadding  (None, 230, 230, 3)  0          ['input_42[0][0]']               
 2D)                                                                                              
                                                                                                  
 conv1/conv (Conv2D)            (None, 112, 112, 64  9408        ['zero_padding2d_54[0][0]']      
                                )                                                          

 conv2_block4_0_relu (Activatio  (None, 56, 56, 160)  0          ['conv2_block4_0_bn[0][0]']      
 n)                                                                                               
                                                                                                  
 conv2_block4_1_conv (Conv2D)   (None, 56, 56, 128)  20480       ['conv2_block4_0_relu[0][0]']    
                                                                                                  
 conv2_block4_1_bn (BatchNormal  (None, 56, 56, 128)  512        ['conv2_block4_1_conv[0][0]']    
 ization)                                                                                         
                                                                                                  
 conv2_block4_1_relu (Activatio  (None, 56, 56, 128)  0          ['conv2_block4_1_bn[0][0]']      
 n)                                                                                               
          

                                                                                                  
 conv3_block2_0_relu (Activatio  (None, 28, 28, 160)  0          ['conv3_block2_0_bn[0][0]']      
 n)                                                                                               
                                                                                                  
 conv3_block2_1_conv (Conv2D)   (None, 28, 28, 128)  20480       ['conv3_block2_0_relu[0][0]']    
                                                                                                  
 conv3_block2_1_bn (BatchNormal  (None, 28, 28, 128)  512        ['conv3_block2_1_conv[0][0]']    
 ization)                                                                                         
                                                                                                  
 conv3_block2_1_relu (Activatio  (None, 28, 28, 128)  0          ['conv3_block2_1_bn[0][0]']      
 n)       

 ization)                                                                                         
                                                                                                  
 conv3_block6_1_relu (Activatio  (None, 28, 28, 128)  0          ['conv3_block6_1_bn[0][0]']      
 n)                                                                                               
                                                                                                  
 conv3_block6_2_conv (Conv2D)   (None, 28, 28, 32)   36864       ['conv3_block6_1_relu[0][0]']    
                                                                                                  
 conv3_block6_concat (Concatena  (None, 28, 28, 320)  0          ['conv3_block5_concat[0][0]',    
 te)                                                              'conv3_block6_2_conv[0][0]']    
                                                                                                  
 conv3_blo

 conv3_block10_concat (Concaten  (None, 28, 28, 448)  0          ['conv3_block9_concat[0][0]',    
 ate)                                                             'conv3_block10_2_conv[0][0]']   
                                                                                                  
 conv3_block11_0_bn (BatchNorma  (None, 28, 28, 448)  1792       ['conv3_block10_concat[0][0]']   
 lization)                                                                                        
                                                                                                  
 conv3_block11_0_relu (Activati  (None, 28, 28, 448)  0          ['conv3_block11_0_bn[0][0]']     
 on)                                                                                              
                                                                                                  
 conv3_block11_1_conv (Conv2D)  (None, 28, 28, 128)  57344       ['conv3_block11_0_relu[0][0]']   
          

...

 on)                                                                                              
                                                                                                  
 conv4_block24_2_conv (Conv2D)  (None, 14, 14, 32)   36864       ['conv4_block24_1_relu[0][0]']   
                                                                                                  
 conv4_block24_concat (Concaten  (None, 14, 14, 1024  0          ['conv4_block23_concat[0][0]',   
 ate)                           )                                 'conv4_block24_2_conv[0][0]']   
                                                                                                  
 pool4_bn (BatchNormalization)  (None, 14, 14, 1024  4096        ['conv4_block24_concat[0][0]']   
                                )                                                                 
                                                                                                  
 pool4_rel

 ization)                                                                                         
                                                                                                  
 conv5_block4_1_relu (Activatio  (None, 7, 7, 128)   0           ['conv5_block4_1_bn[0][0]']      
 n)                                                                                               
                                                                                                  
 conv5_block4_2_conv (Conv2D)   (None, 7, 7, 32)     36864       ['conv5_block4_1_relu[0][0]']    
                                                                                                  
 conv5_block4_concat (Concatena  (None, 7, 7, 640)   0           ['conv5_block3_concat[0][0]',    
 te)                                                              'conv5_block4_2_conv[0][0]']    
                                                                                                  
 conv5_blo

 conv5_block8_concat (Concatena  (None, 7, 7, 768)   0           ['conv5_block7_concat[0][0]',    
 te)                                                              'conv5_block8_2_conv[0][0]']    
                                                                                                  
 conv5_block9_0_bn (BatchNormal  (None, 7, 7, 768)   3072        ['conv5_block8_concat[0][0]']    
 ization)                                                                                         
                                                                                                  
 conv5_block9_0_relu (Activatio  (None, 7, 7, 768)   0           ['conv5_block9_0_bn[0][0]']      
 n)                                                                                               
                                                                                                  
 conv5_block9_1_conv (Conv2D)   (None, 7, 7, 128)    98304       ['conv5_block9_0_relu[0][0]']    
          

 on)                                                                                              
                                                                                                  
 conv5_block13_1_conv (Conv2D)  (None, 7, 7, 128)    114688      ['conv5_block13_0_relu[0][0]']   
                                                                                                  
 conv5_block13_1_bn (BatchNorma  (None, 7, 7, 128)   512         ['conv5_block13_1_conv[0][0]']   
 lization)                                                                                        
                                                                                                  
 conv5_block13_1_relu (Activati  (None, 7, 7, 128)   0           ['conv5_block13_1_bn[0][0]']     
 on)                                                                                              
                                                                                                  
 conv5_blo

__________________________________________________________________________________________________


In [ ]:
history = model.fit_generator(train_generator,validation_data=valid_generator,epochs=30,callbacks=callbacks_lst)